In [ ]:
import os
import pandas as pd
from glob import glob
import tensorflow as tf
import json
from pathlib import Path
import numpy as np
from multiprocessing import Pool, cpu_count
from tqdm import tqdm

MODEL = '/kaggle/input/pp21models/A_baseline_noaug.fold_1'

paths = glob('/kaggle/input/**/test_images/*.jpg', recursive=True)
files = list(map(os.path.basename, paths))

model = tf.keras.models.load_model(f'{MODEL}.h5')
meta = json.loads(Path(f'{MODEL}.meta.json').read_text())

def _path_to_x(path):
    x = tf.io.read_file(path)
    x = tf.image.decode_jpeg(x, channels=3)
    x = tf.image.resize(
        x,
        tuple(meta['image_size'][:2]),
        method=tf.image.ResizeMethod.BICUBIC,
    ).numpy().astype(np.uint8)
    return x

def _y_to_labels(y):
    labels = [] 
    
    for i, v in enumerate(y):
        if v > 0.5:
            labels.append(meta['classes'][i])
    
    if len(labels) == 0:
        labels=['healthy']
    
    return ' '.join(labels)

labels = []

for path in paths:
    x = _path_to_x(path)
    y = model.predict(np.array([x]), verbose=0)[0]
    labels.append(_y_to_labels(y))

dfs = pd.DataFrame()
dfs.insert(0, 'image', files)
dfs.insert(1, 'labels', labels)
dfs.to_csv('/kaggle/working/submission.csv', index=False)

In [ ]:
# ! cat /kaggle/working/submission.csv